In [2]:
import functions0 as f0
import numpy as np
import datetime
import qiskit.ignis.mitigation.measurement as mc
from qiskit import Aer, assemble, QuantumCircuit, QuantumRegister, ClassicalRegister, IBMQ, transpile, execute
from qiskit.providers.aer import AerSimulator, QasmSimulator
from qiskit.opflow import Zero, One, I, X, Y, Z
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity
import matplotlib.pyplot as plt

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')

sim_noisy_jakarta = QasmSimulator.from_backend(provider.get_backend('ibmq_jakarta'))
backend_sim_jakarta = sim_noisy_jakarta 
backend = provider.get_backend('ibmq_jakarta')
backend_sim = Aer.get_backend('qasm_simulator')

ibmqfactory.load_account:WARNING:2022-03-17 22:48:29,594: Credentials are already in use. The existing account in the session will be replaced.


## IBM open-science-prize-2021/22 solution. By Quantum Polo Gang: Ruben, Fabio & Valerio

## Decomposition:
- We computed numerically the operator of N trotter steps, for a certain evolution time: $U^n$
- Observing that this operator preserves the magnetization of the system, if the initial state has a fixed magnetization (so, it isn't a superposition of state with different magnetization) is possible to decompose the operator with 4 c-not. If the initial state is a superposition of states with different magnetization the best decomposition we found has 11 c-not (14 for the Jakarta geometry).
- Our initial state is $|110>$ (qubits 5,3 and 1 respectively) so we can use the best decomposition (4 c-not).

To see the decomposition procedure go in decomposition.ipynb file.

Let's start from the defining of the evolution cirquit:

In [3]:
steps=42
time=np.pi
initial_state="110"

qr, qc = f0.evolution_cirquit(steps=steps, time=time, initial_state="110") #DEVO IMPLEMENTARLA PER QUALSIASI STATO INIZIALE!
qc.draw()

q_0: ───────────────────
          ┌─────┐       
q_1: ─────┤0    ├───────
          │     │       
q_2: ─────┤  M1 ├───────
     ┌───┐│     │┌─────┐
q_3: ┤ X ├┤1    ├┤0    ├
     └───┘└─────┘│     │
q_4: ────────────┤  M2 ├
     ┌───┐       │     │
q_5: ┤ X ├───────┤1    ├
     └───┘       └─────┘
q_6: ───────────────────

Then we add the copy check for the mitigation

In [4]:
qc = f0.add_check(qc, [qr[1],qr[3],qr[5]], [qr[0],qr[2],qr[4],qr[6]], type="4copy_check")
qc.draw()

┌──────────────┐
q_0: ───────────────────┤0             ├
          ┌─────┐       │              │
q_1: ─────┤0    ├───────┤4             ├
          │     │       │              │
q_2: ─────┤  M1 ├───────┤1             ├
     ┌───┐│     │┌─────┐│              │
q_3: ┤ X ├┤1    ├┤0    ├┤5 4copy_check ├
     └───┘└─────┘│     ││              │
q_4: ────────────┤  M2 ├┤2             ├
     ┌───┐       │     ││              │
q_5: ┤ X ├───────┤1    ├┤6             ├
     └───┘       └─────┘│              │
q_6: ───────────────────┤3             ├
                        └──────────────┘

Then, we build the tomografy cirquits

In [5]:
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter

qcs = state_tomography_circuits(qc, [qr[1],qr[3],qr[5]])
qcs_na = state_tomography_circuits(qc, [qr[1],qr[3],qr[5]])

for qc in qcs:
    cr_anc = ClassicalRegister(4)
    qc.add_register(cr_anc)
    qc.barrier()
    qc.measure([0,2,4,6],cr_anc)

qcs[10].draw()

┌──────────────┐ ░                         ░ ┌─┐   »
  q_0: ───────────────────┤0             ├─░─────────────────────────░─┤M├───»
            ┌─────┐       │              │ ░ ┌─────┐┌───┐     ┌─┐    ░ └╥┘   »
  q_1: ─────┤0    ├───────┤4             ├─░─┤ Sdg ├┤ H ├─────┤M├────░──╫────»
            │     │       │              │ ░ └─────┘└───┘     └╥┘    ░  ║ ┌─┐»
  q_2: ─────┤  M1 ├───────┤1             ├─░───────────────────╫─────░──╫─┤M├»
       ┌───┐│     │┌─────┐│              │ ░  ┌───┐  ┌─┐       ║     ░  ║ └╥┘»
  q_3: ┤ X ├┤1    ├┤0    ├┤5 4copy_check ├─░──┤ H ├──┤M├───────╫─────░──╫──╫─»
       └───┘└─────┘│     ││              │ ░  └───┘  └╥┘       ║     ░  ║  ║ »
  q_4: ────────────┤  M2 ├┤2             ├─░──────────╫────────╫─────░──╫──╫─»
       ┌───┐       │     ││              │ ░ ┌─────┐  ║  ┌───┐ ║ ┌─┐ ░  ║  ║ »
  q_5: ┤ X ├───────┤1    ├┤6             ├─░─┤ Sdg ├──╫──┤ H ├─╫─┤M├─░──╫──╫─»
       └───┘       └─────┘│              │ ░ └─────┘  ║  └───┘ ║ └╥┘ ░  ║  ║ »
  q_6: ───────────────────┤3             ├─░──────────╫────────╫──╫──░──╫──╫─»
                          └──────────────┘ ░          ║        ║  ║  ░  ║  ║ »
 c0: 3/═══════════════════════════════════════════════╩════════╩══╩═════╬══╬═»
                                                      1        0  2     ║  ║ »
c12: 4/═════════════════════════════════════════════════════════════════╩══╩═»
                                                                        0  1 »
«             
«  q_0: ──────
«             
«  q_1: ──────
«             
«  q_2: ──────
«             
«  q_3: ──────
«       ┌─┐   
«  q_4: ┤M├───
«       └╥┘   
«  q_5: ─╫────
«        ║ ┌─┐
«  q_6: ─╫─┤M├
«        ║ └╥┘
« c0: 3/═╬══╬═
«        ║  ║ 
«c12: 4/═╩══╩═
«        2  3

Building the calibration cirquits

In [6]:
qcs_calibs, meas_calibs = f0.calibration_cirquits("column_evolution_remake", q_anc=[0,2,4,6], check="yes", check_type="4copy_check")
state_lables = f0.bin_list(7)

Than we run all the cirquits

In [11]:
backend_aus = backend
shots = 32000

job_evo=execute(qcs, backend=backend_aus, shots=shots)
#job_cal=execute(meas_calibs, backend=backend_aus, shots=shots)
job_cal_our=execute(qcs_calibs, backend=backend_aus, shots=shots)

or we can retrieve the jobs

In [13]:
evo_ID = "6233ae39d97bff04d66929e9"
cal_ID = "6233ae3ba2f72dff43da994f"

evo_job=backend.retrieve_job(evo_ID)
job_cal_our=backend.retrieve_job(cal_ID)

In [14]:
#meas_fitter = mc.CompleteMeasFitter(job_cal.result(), state_labels=state_lables)
meas_fitter_our = mc.CompleteMeasFitter(job_cal_our.result(), state_labels=state_lables)

target_state = (One^One^Zero).to_matrix()

fids=np.zeros([1,4])
#fids_mean=np.zeros(4)
#fids_dev=np.zeros(4)

for j in range(1):
    
    res = job_evo.result()

    #new_res, new_res_nm = f0.mitigate(res, Measure_Mitig="yes", ancillas_conditions=['0011','1110','1101'], meas_fitter=meas_fitter)
    new_res_our, new_res_nm = f0.mitigate(res, Measure_Mitig="yes", ancillas_conditions=['0011','1110','1101'], meas_fitter=meas_fitter_our)
    new_res_not_mitigated = f0.mitigate(res, Measure_Mitig="no", ancillas_conditions=f0.bin_list(4))

    fids[j,0] = f0.fidelity_count(new_res_not_mitigated, qcs_na, target_state)
    fids[j,1] = f0.fidelity_count(new_res_nm, qcs_na, target_state)
    #fids[j,2] = f0.fidelity_count(new_res, qcs_na, target_state)
    fids[j,3] = f0.fidelity_count(new_res_our, qcs_na, target_state)

for i in range(4):
    #fids_mean[i]=np.mean(fids[:,i])
    #fids_dev[i]=np.std(fids[:,i])

    #fids_meanY[i]=np.mean(fidsY[:,i])
    #fids_devY[i]=np.std(fidsY[:,i])
    print(fids[0,i])

0.8097005029135075
0.8628857599813681
0.0
0.9922172624447446
